# Spanish Electricity Pool Forecasting: Import Data from ESIOS

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from fext import esios

In [ ]:
import config

sns.set(
    rc=config.set_plot_features(), style="darkgrid"
       )

TOKEN = config.ESIOS_API_KEY
PATH = "./data/"

In [ ]:
START_DATE="2014-01-01"
END_DATE = dt.datetime.now().date().strftime("%Y-%m-%d")

start_date_suffix = START_DATE.replace("-","_")
end_date_suffix = END_DATE.replace("-","_")

END_DATE

### Test ESIOS API to retrieve indicators metadata

In [ ]:
import requests
import json

url = "https://api.esios.ree.es/indicators/"

headers = {'content-Type': 'application/json', 'Authorization': 'Token token={}'.format(TOKEN)}

response = requests.get(url, headers=headers, stream=True)
print(response)

In [ ]:
data_dict0 = json.loads(response.content.decode('UTF-8'))
type(data_dict0)

In [ ]:
indicators_df = pd.DataFrame(data_dict0['indicators'])
indicators_df.head()

In [ ]:
indicators_df[["id","name","description"]].to_excel("esios_indicators.xlsx", index=False)

### Query ESIOS API to retrieve aggregated data from one indicator

**Download daily average SPOT price**

In [ ]:
_, data_dict, data_df = esios.get_esios_data(token=TOKEN,
                                       indicator = 600,
                    start_dt= '2019-05-31',
                    end_dt= '2019-06-03',
                    period = 'day',
                    agg= "average",
                    dev=False
                                      )


In [ ]:
data_df

### Query ESIOS API to retrieve aggregated data from multiple indicators

**Basic example**

In [ ]:
multi_data_dict, multi_data_df = esios.get_esios_multi_data(TOKEN, period='day')

In [ ]:
multi_data_df

### Download and build a dataset with market pool price and generation share by technology

**Get indicators list and split it by aggregating function**

In [ ]:
tickers_df = pd.read_excel('esios_ind_retrieve_list.xlsx')
tickers_df

In [ ]:
indicators_retr_sum = list(tickers_df["id"].unique())[1:-3]

indicators_retr_sum

In [ ]:
indicators_retr_avg = list(tickers_df["id"].unique())[0]
indicators_retr_avg

**Download and index price data**

In [ ]:
data_price_14_19_dict, data_price_14_19_df = esios.get_esios_multi_data(token=TOKEN,
                    indicators = [600],
                    start_dt = START_DATE,
                    end_dt = END_DATE,
                    period = 'day',
                    agg = 'average',
                    dev=False
                  )

In [ ]:
data_price_14_19_df["date"] = pd.to_datetime(data_price_14_19_df["datetime"].str[:10],infer_datetime_format=True).dt.date
data_price_14_19_df.index= data_price_14_19_df["date"]
data_price_14_19_df.head()

In [ ]:
data_price_14_19_df.tail()

**Download and index P48 generation data**

In [ ]:
data_gen_14_19_dict, data_gen_14_19_df = esios.get_esios_multi_data(token=TOKEN,
                    indicators = indicators_retr_sum,
                    start_dt = START_DATE,
                    end_dt = END_DATE,
                    period = 'day',
                    agg = 'sum',
                    dev=False
                  )

In [ ]:
data_gen_14_19_df["date"] = pd.to_datetime(data_gen_14_19_df["datetime"].str[:10],infer_datetime_format=True).dt.date
data_gen_14_19_df.index = data_gen_14_19_df["date"]
data_gen_14_19_df.head()

In [ ]:
data_gen_14_19_df.tail()

**Merge both datasets and write to csv**

In [ ]:
data_14_19_df = pd.concat([data_price_14_19_df,data_gen_14_19_df], axis=0)
data_14_19_df.info()

In [ ]:
with open(PATH + 'raw_esios_price_daily.json', 'w') as json_file:
    json.dump(data_price_14_19_dict, json_file)

In [ ]:
with open(PATH + 'raw_esios_gen_daily.json', 'w') as json_file:
    json.dump(data_gen_14_19_dict, json_file)

In [ ]:
data_gen_14_19_df.info()


In [ ]:
select_columns= ['date', 'datetime', 'datetime_utc', 'geo_id', 'geo_name','tz_time', 'value', 'indicador', 'idx']

data_14_19_df[select_columns].to_csv(PATH + "esios_daily.csv", index=False)

In [ ]:
gen = data_14_19_df[
    data_14_19_df["indicador"]=="Generación programada PBF total"][["value"]]
gen["mave_52"] = gen["value"].rolling(52).mean()
gen.plot()
plt.title("Daily P48 Total Generation")
plt.ylabel("MWh")
plt.show()

In [ ]:
price = data_14_19_df[
    (data_14_19_df["indicador"]=="Precio mercado SPOT Diario") &
    (data_14_19_df["geo_id"]==3)][["value"]]
price["mave_52"] = price["value"].rolling(52).mean()
price.plot()

plt.title("Daily SPOT price")
plt.ylabel("€/MWh")
plt.show()


In [ ]:
sns.jointplot(x=gen.values, y=price.values, kind="reg")
plt.xlabel("P48 Total Generation (MWh)")
plt.ylabel("Daily SPOT price (€/MWh)")
plt.show()